In [2]:
%load_ext google.cloud.bigquery
%matplotlib inline
from google.cloud import bigquery
client = bigquery.Client()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests

In [2]:
def create_sql(database, table):
    return '''
            SELECT
                *
            FROM `{}.{}`
            LIMIT 15
            '''.format(database, table)




### transactions

In [95]:
database = 'bigquery-public-data.ethereum_blockchain'
table = 'transactions'
sql = create_sql(database, table)

df = client.query(sql).to_dataframe()
df.head()['hash'][0]

'0x1d023549fae00a06c5aa993846894c31f6278f8941890d606ac68634aac032f0'

In [123]:
database = 'bigquery-public-data.ethereum_blockchain'
table = 'blocks'
sql = create_sql(database, table)

df = client.query(sql).to_dataframe()

Creating a new dataset on Google Cloud to store toy data

In [136]:
dataset_id='transactions_subset'
dataset = client.create_dataset(dataset_id)

In [137]:
TABLE = 'subset'
job_config = bigquery.QueryJobConfig()
# Set the destination table
table_ref = client.dataset(dataset_id).table(TABLE)
job_config.destination = table_ref

sql = """
    SELECT *
    FROM `bigquery-public-data.ethereum_blockchain.transactions`
    ORDER BY block_timestamp DESC
    LIMIT 20000;
"""

# Start the query, passing in the extra configuration.
query_job = client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query
    # and of the destination table.
    location='US',
    job_config=job_config)  # API request - starts the query

query_job.result()  # Waits for the query to finish
print('Query results loaded to table {}'.format(table_ref.path))

Query results loaded to table /projects/eth-tokens/datasets/transactions_subset/tables/subset


In [ ]:
%%bigquery

subset transactions table to manageable number of rows: 20,000? Trying with most recent

In [6]:
%%bigquery
WITH tot AS
    (WITH subset AS(SELECT *
                FROM `bigquery-public-data.ethereum_blockchain.transactions`
                ORDER BY block_timestamp DESC
                LIMIT 20000
                   ),

    f AS(SELECT DISTINCT from_address, count(*) as num_outgoing_txns
        FROM subset
        GROUP BY from_address),
    t AS(SELECT DISTINCT to_address, count(*) as num_incoming_txns
        FROM subset
        GROUP BY to_address)
    SELECT COALESCE(from_address,to_address) as addr,
            COALESCE(num_outgoing_txns,0) as outgoing_txns,
            COALESCE(num_incoming_txns,0) as incoming_txns
    FROM f FULL JOIN t
    ON f.from_address = t.to_address)

SELECT *, (outgoing_txns + incoming_txns) as total_txns
FROM tot
ORDER BY total_txns desc
LIMIT 50


,addr,outgoing_txns,incoming_txns,total_txns
0,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1628,0,1628
1,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,804,0,804
2,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,0,460,460
3,0x5892aad75a615014aa88c4b621caff1fe45dc850,0,387,387
4,0xa7a7899d944fe658c4b0a1803bab2f490bd3849e,373,0,373
5,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,89,252,341
6,0xe8780b48bdb05f928697a5e8155f672ed91462f7,0,295,295
7,0xad3ee1abb108e0a18b43b41632a23b64e9121ac3,278,0,278
8,0x29d31eb86dd7cabb2d00f6a62eecfa1cf6e33135,0,276,276
9,0x1a7a8bd9106f2b8d977e08582dc7d24c723ab0db,0,275,275


need to default to zero when no transactions are present

In [4]:
%%bigquery
SELECT from_address, nonce 
FROM `bigquery-public-data.ethereum_blockchain.transactions`
WHERE from_address = '0xea674fdde714fd979de3edf0f56aa9716b898ec8'
ORDER BY nonce desc
LIMIT 5

,from_address,nonce
0,0xea674fdde714fd979de3edf0f56aa9716b898ec8,16425925
1,0xea674fdde714fd979de3edf0f56aa9716b898ec8,16425924
2,0xea674fdde714fd979de3edf0f56aa9716b898ec8,16425923
3,0xea674fdde714fd979de3edf0f56aa9716b898ec8,16425922
4,0xea674fdde714fd979de3edf0f56aa9716b898ec8,16425921


In [ ]:
%%bigquery
SELECT from_address, nonce 
FROM `bigquery-public-data.ethereum_blockchain.transactions`
WHERE from_address = '0xea674fdde714fd979de3edf0f56aa9716b898ec8'
ORDER BY nonce desc
LIMIT 5

In [7]:
%%bigquery
WITH tot AS
    (WITH subset AS(SELECT *
                FROM `bigquery-public-data.ethereum_blockchain.transactions`
                   ),

    f AS(SELECT DISTINCT from_address, max(nonce) as maxnonce, count(*) as num_outgoing_txns
        FROM subset
        GROUP BY from_address),
    t AS(SELECT DISTINCT to_address, count(*) as num_incoming_txns
        FROM subset
        GROUP BY to_address)
    SELECT COALESCE(from_address,to_address) as addr,
            COALESCE(num_outgoing_txns,0) as outgoing_txns,
            COALESCE(num_incoming_txns,0) as incoming_txns,
            maxnonce
    FROM f FULL JOIN t
    ON f.from_address = t.to_address)

SELECT *, (outgoing_txns + incoming_txns) as total_txns
FROM tot
ORDER BY total_txns desc
LIMIT 50

,addr,outgoing_txns,incoming_txns,maxnonce,total_txns
0,0xea674fdde714fd979de3edf0f56aa9716b898ec8,16425926,359,16425925.0,16426285
1,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,0,10520475,NaN,10520475
2,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,10164702,368,10164701.0,10165070
3,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,2762530,4642263,2762529.0,7404793
4,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,6814854,235407,6814853.0,7050261
5,0x829bd824b016326a401d083b33d092293333a830,6623903,4050,6623902.0,6627953
6,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,0,5159946,NaN,5159946
7,0x5a0b54d5dc17e0aadc383d2db43b0a0d3e029c4c,4608447,31,4608446.0,4608478
8,0x2a65aca4d5fc5b5c859090a6c34d164135398226,3906821,28,3906820.0,3906849
9,0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0,0,2954243,NaN,2954243


Load prices csv into bigquery

In [ ]:
dataset_id='transactions_subset'
dataset = client.get_dataset(dataset_id)
TABLE = 'prices'
job_config = bigquery.QueryJobConfig()
# Set the destination table
table_ref = client.dataset(dataset_id).table(TABLE)
job_config.destination = table_ref

sql = """
    SELECT *
    FROM `bigquery-public-data.ethereum_blockchain.transactions`
    ORDER BY block_timestamp DESC
    LIMIT 20000;
"""

# Start the query, passing in the extra configuration.
query_job = client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query
    # and of the destination table.
    location='US',
    job_config=job_config)  # API request - starts the query

query_job.result()  # Waits for the query to finish
print('Query results loaded to table {}'.format(table_ref.path))

In [9]:
client.get_dataset?

Signature: client.get_dataset(dataset_ref, retry=<google.api_core.retry.Retry object at 0x109bfdcc0>)
Docstring:
Fetch the dataset referenced by ``dataset_ref``

Args:
    dataset_ref (Union[                 :class:`~google.cloud.bigquery.dataset.DatasetReference`,                 str,             ]):
        A reference to the dataset to fetch from the BigQuery API.
        If a string is passed in, this method attempts to create a
        dataset reference from a string using
        :func:`~google.cloud.bigquery.dataset.DatasetReference.from_string`.
    retry (:class:`google.api_core.retry.Retry`):
        (Optional) How to retry the RPC.

Returns:
    google.cloud.bigquery.dataset.Dataset:
        A ``Dataset`` instance.
File:      ~/anaconda3/lib/python3.6/site-packages/google/cloud/bigquery/client.py
Type:      method


In [ ]:
%%bigquery
SELECT *
FROM `bigquery-public-data.ethereum_blockchain.transactions`
WHERE 
ORDER BY nonce desc
LIMIT 5